In [1]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model

from keras.activations import softmax
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *
from keras.models import model_from_json
import json



Using TensorFlow backend.


In [2]:
def sanitize(string):
    words = string.split(' ')
    return words

In [3]:
!nvidia-smi


Mon Jul 22 07:31:16 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   47C    P8     8W / 200W |    535MiB /  8118MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
#read csv

train = pd.read_csv("../dataset/Phrase_Level/PAWS_Wiki/LABELED_PAWS_Wiki_TRAIN.tsv", sep='\t')

dev = pd.read_csv("../dataset/Phrase_Level/PAWS_Wiki/LABELED_PAWS_Wiki_DEV.tsv", sep='\t')

test = pd.read_csv("../dataset/Phrase_Level/PAWS_Wiki/LABELED_PAWS_Wiki_TEST.tsv", sep='\t')


print(train[:5])

train['title1_tokenized'] = \
    train.loc[:, 'sent1'] \
         .apply(sanitize)
train['title2_tokenized'] = \
    train.loc[:, 'sent2'] \
         .apply(sanitize)


dev['title1_tokenized'] = \
    dev.loc[:, 'sent1'] \
         .apply(sanitize)
dev['title2_tokenized'] = \
    dev.loc[:, 'sent2'] \
         .apply(sanitize)


   TRAIN_PAWS_Wiki_id   PPDB_id  \
0                21.0  160456.0   
1                21.0  243017.0   
2                33.0  395059.0   
3                33.0  395061.0   
4                33.0  395071.0   

                                               sent1  \
0  For their performances in the game , quarterba...   
1  For their performances in the game , quarterba...   
2  The leaves are generally 1.5-4 mm long and 0.2...   
3  The leaves are generally 1.5-4 mm long and 0.2...   
4  The leaves are generally 1.5-4 mm long and 0.2...   

                                               sent2      paraphrase1  \
0  Quarterback P. J. Williams and Defensive Back ...   most valuable    
1  Quarterback P. J. Williams and Defensive Back ...         's most    
2  The leaves are usually 1.5-4 mm long and 0.2-0...   are generally    
3  The leaves are usually 1.5-4 mm long and 0.2-0...   are generally    
4  The leaves are usually 1.5-4 mm long and 0.2-0...   are generally    

     paraphra

In [165]:
train.labels1[:4]

0    O O O O O O O O O O O O O O O O O O O O O B E O O
1    O O O O O O O O O O O O O O O O O O O O B E O O O
2                              O O B E O O O O O O O O
3                              O O B E O O O O O O O O
Name: labels1, dtype: object

In [5]:
train.head()

,TRAIN_PAWS_Wiki_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,label,labels1,labels2,title1_tokenized,title2_tokenized
0,21.0,160456.0,"For their performances in the game , quarterba...",Quarterback P. J. Williams and Defensive Back ...,most valuable,valuable,0,O O O O O O O O O O O O O O O O O O O O O B E O O,O O O O O O O O O O O O O S O O O O O O O O O O O,"[For, their, performances, in, the, game, ,, q...","[Quarterback, P., J., Williams, and, Defensive..."
1,21.0,243017.0,"For their performances in the game , quarterba...",Quarterback P. J. Williams and Defensive Back ...,'s most,most,0,O O O O O O O O O O O O O O O O O O O O B E O O O,O O O O O O O O O O O O S O O O O O O O O O O O O,"[For, their, performances, in, the, game, ,, q...","[Quarterback, P., J., Williams, and, Defensive..."
2,33.0,395059.0,The leaves are generally 1.5-4 mm long and 0.2...,The leaves are usually 1.5-4 mm long and 0.2-0...,are generally,are usually,1,O O B E O O O O O O O O,O O B E O O O O O O O O,"[The, leaves, are, generally, 1.5-4, mm, long,...","[The, leaves, are, usually, 1.5-4, mm, long, a..."
3,33.0,395061.0,The leaves are generally 1.5-4 mm long and 0.2...,The leaves are usually 1.5-4 mm long and 0.2-0...,are generally,are usually,1,O O B E O O O O O O O O,O O B E O O O O O O O O,"[The, leaves, are, generally, 1.5-4, mm, long,...","[The, leaves, are, usually, 1.5-4, mm, long, a..."
4,33.0,395071.0,The leaves are generally 1.5-4 mm long and 0.2...,The leaves are usually 1.5-4 mm long and 0.2-0...,are generally,are usually,1,O O B E O O O O O O O O,O O B E O O O O O O O O,"[The, leaves, are, generally, 1.5-4, mm, long,...","[The, leaves, are, usually, 1.5-4, mm, long, a..."


In [5]:
embeddings_index = {}
f = open('../glove.840B.300d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    embeddings_index[word] = coefs

f.close()

print('GloVe data loaded')

GloVe data loaded


In [193]:
#tokenize sentences

def normalize_and_split(sentence):
    s=sentence.split()
    l=[x.lower() for x in s]
    
    return l

def normalize_and_split2(sentence):
    s=sentence.split()
    l=[x.lower() for x in s]
    first_word=(l[0])[2:]
    l[0]=first_word
    return l

#tokenize labels

def t_label(column, max_length):
    
    ret=[]
  
    for y in column:   #each list
        
        s = y.split() #split into individual alphabets
        temp=[]
        
        for x in s:
            temp.append(label_to_index[x])
             
    
        for _ in range(max_length-len(s)): 
            temp.append(1)
        
        temp=np.asarray(temp)
        
        y_train = keras \
            .utils \
            .to_categorical(temp, 5)    #do one hot on each word
        
        
        
        ret.append(y_train)
           
    return list(ret)

label_to_index = {
    'O': 0, 
    'B': 1, 
    'I': 2,
    'E': 3,
    'S': 4
}

In [110]:
normalize_and_split("I like to eateat piE")

['i', 'like', 'to', 'eateat', 'pie']

In [65]:
normalize_and_split("b'I Am LiKINg iTt")

['i', 'am', 'liking', 'itt']

In [7]:
train_Tb=Table.read_table("../dataset/Phrase_Level/PAWS_Wiki/LABELED_PAWS_Wiki_TRAIN.tsv")
dev_Tb=Table.read_table("../dataset/Phrase_Level/PAWS_Wiki/LABELED_PAWS_Wiki_DEV.tsv")
test_Tb=Table.read_table("../dataset/Phrase_Level/PAWS_Wiki/LABELED_PAWS_Wiki_TEST.tsv")

train_Tb_Q=Table.read_table("../dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_TRAIN.tsv")
dev_Tb_Q=Table.read_table("../dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_DEV.tsv")

dev_Tb_S=Table.read_table("../dataset/Phrase_Level/SNLI/SNLI_PPDB_DEV.tsv")
test_Tb_S=Table.read_table("../dataset/Phrase_Level/SNLI/PPDB_SNLI_TEST.tsv")



In [8]:
def keep_one_para_pair(Tb):   #takes in origin table, keeps only one pair of paraphrase per sentence, and then return the modified table
    
    to_drop=[]
    pair_id=Tb.column(0)
    for x in range(Tb.num_rows-1):
        if pair_id[x]==pair_id[x+1]:
            to_drop.append(x)
    new_Tb=Tb.exclude(to_drop)
    return new_Tb

In [9]:
train_PAWS_Wiki=keep_one_para_pair(train_Tb)   #2739

dev_PAWS_Wiki=keep_one_para_pair(dev_Tb)     #924

test_PAWS_Wiki=keep_one_para_pair(test_Tb)   #1191

train_PAWS_QQP=keep_one_para_pair(train_Tb_Q)  #791

dev_PAWS_QQP=keep_one_para_pair(dev_Tb_Q)     #63

dev_SNLI=keep_one_para_pair(dev_Tb_S)       #308   

test_SNLI=keep_one_para_pair(test_Tb_S)       #229


In [11]:
test_SNLI.show(4)

SNLI_PairID,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,gold_label,labels1,labels2
6526219567.jpg#4r1e,1.10807e+06,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to,seem to,entailment,O O O O O O O B E O O O,O O O O O O O O B E O O O
4460943467.jpg#0r1e,302929,3 young man in hoods standing in the middle of a quiet s ...,Three hood wearing people stand in a street.,standing in,stand in,entailment,O O O O O B E O O O O O O O O O,O O O O B E O O
3036382555.jpg#0r1n,1.00273e+06,"The two young girls are dressed as fairies, and are play ...",The two girls play in the Autumn.,playing in,play in,neutral,O O O O O O O O O O B E O O O,O O O B E O O
4479330885.jpg#3r1e,958598,Several younger people sitting in front of a statue.,several young people sitting outside,younger people,young,neutral,O B E O O O O O O,O S O O O


In [194]:
def help_norm(Tb):
     
    
    if Tb is train_PAWS_QQP or Tb is dev_PAWS_QQP:
        a=Tb.apply(normalize_and_split2, "sent1")
        b=Tb.apply(normalize_and_split2, "sent2")
        
    else:
        a=Tb.apply(normalize_and_split, "sent1")
        b=Tb.apply(normalize_and_split, "sent2")    
        
       
    
    
    c=t_label(Tb.column("labels1"), 58)
    d=t_label(Tb.column("labels2"), 58)
    
   

    
    return list(a),list(b),list(c),list(d)

[2, 4]

In [195]:
#tokenize sentences, and one-hot labels

TRPWS1, TRPWS2, TRPWL1, TRPWL2=help_norm(train_PAWS_Wiki)   #training paws wiki sentence 1, btw, and the rest are named in a similar fashion

DPWS1, DPWS2,DPWL1, DPWL2 = help_norm(dev_PAWS_Wiki)

TEPWS1, TEPWS2, TEPWL1, TEPWL2=help_norm(test_PAWS_Wiki)

TRPQS1, TRPQS2, TRPQL1, TRPQL2=help_norm(train_PAWS_QQP)

DPQS1, DPQS2, DPQL1, DPQL2=help_norm(dev_PAWS_QQP)

DSS1, DSS2, DSL1, DSL2 = help_norm(dev_SNLI)

TESS1, TESS2, TESL1, TESL2=help_norm(test_SNLI)

#also prints out maximum sentence length for each dataset


In [15]:
TRPW=create_Tb(train_PAWS_Wiki, TRPWS1, TRPWS2, TRPWL1, TRPWL2)
DPW=create_Tb(dev_PAWS_Wiki, DPWS1, DPWS2, DPWL1, DPWL2)
TEPW=create_Tb(test_PAWS_Wiki, TEPWS1, TEPWS2, TEPWL1, TEPWL2)
TRPQ=create_Tb(train_PAWS_QQP, TRPQS1, TRPQS2, TRPQL1, TRPQL2)
DPQ=create_Tb(dev_PAWS_QQP, DPQS1, DPQS2, DPQL1, DPQL2)
DS=create_Tb(dev_SNLI, DSS1, DSS2, DSL1, DSL2)
TES=create_Tb(test_SNLI, TESS1, TESS2, TESL1, TESL2)
#max seq length say 58

NameError: name 'create_Tb' is not defined

In [12]:
def create_Tb(Tb, s1, s2, l1, l2):         #visualize tokenization with a table
    zz=Tb.with_columns("sent1_token", s1, "sent2_token", s2, "label1_token", l1, "label2_token", l2)
    return zz

In [194]:
TRPW.show(4)


TRAIN_PAWS_Wiki_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,label,labels1,labels2,sent1_token,sent2_token,label1_token,label2_token
21,243017,"For their performances in the game , quarterback Jameis ...",Quarterback P. J. Williams and Defensive Back Jameis Win ...,'s most,most,0,O O O O O O O O O O O O O O O O O O O O B E O O O,O O O O O O O O O O O O S O O O O O O O O O O O O,"['for', 'their', 'performances', 'in', 'the', 'game', ', ...","['quarterback', 'p.', 'j.', 'williams', 'and', 'defensiv ...",[[1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [ ...,[[1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [ ...
33,395099,The leaves are generally 1.5-4 mm long and 0.2-0.7 mm wide .,The leaves are usually 1.5-4 mm long and 0.2-0.7 mm wide .,are generally,are usually,1,O O B E O O O O O O O O,O O B E O O O O O O O O,"['the', 'leaves', 'are', 'generally', '1.5-4', 'mm', 'lo ...","['the', 'leaves', 'are', 'usually', '1.5-4', 'mm', 'long ...",[[1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [0. 1. 0. 0. 0.] [ ...,[[1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [0. 1. 0. 0. 0.] [ ...
63,100574,She has taken part in the second season of the most popu ...,She has participated in the second season of the most co ...,taken part in,participated in,0,O O B I E O O O O O O O O O O O O O O O O O O O O,O O B E O O O O O O O O O O O O O O O O O O O,"['she', 'has', 'taken', 'part', 'in', 'the', 'second', ' ...","['she', 'has', 'participated', 'in', 'the', 'second', 's ...",[[1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [0. 1. 0. 0. 0.] [ ...,[[1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [0. 1. 0. 0. 0.] [ ...
86,404940,"Lemmings , by contrast , are conspicuously colored and b ...","By contrast , the lemmings are strikingly colored and be ...",even human,human,1,O O O O O O O O O O O O O O B E O O,O O O O O O O O O O O O O O O S O O,"['lemmings', ',', 'by', 'contrast', ',', 'are', 'conspic ...","['by', 'contrast', ',', 'the', 'lemmings', 'are', 'strik ...",[[1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [ ...,[[1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [1. 0. 0. 0. 0.] [ ...


25

In [196]:
train_corpus=TRPWS1+TRPWS2+DPWS1+DPWS2+TEPWS1+TEPWS2+TRPQS1+TRPQS2+DPQS1+DPQS2

dev_corpus=DSS1 +DSS2+ TESS1+ TESS2

['an',
 'asian',
 'woman',
 'sitting',
 'outside',
 'an',
 'outdoor',
 'market',
 'stall.']

In [358]:
len(train_sen1)

5708

In [197]:
train_sen1=TRPWS1+DPWS1+TEPWS1+TRPQS1+DPQS1

train_sen2=TRPWS2+DPWS2+TEPWS2+TRPQS2+DPQS2

train_label1=np.array(TRPWL1+DPWL1+TEPWL1+TRPQL1+DPQL1)

train_label2=np.array(TRPWL2+DPWL2+TEPWL2+TRPQL2+DPQL2)

dev_sen1=DSS1+TESS1
dev_sen2=DSS2+TESS2
dev_label1=np.array(DSL1+TESL1)
dev_label2=np.array(DSL2 + TESL2)


In [69]:
train_label1[2]           #10000: O, 01000: B, 00100: I, 00010: E, 00001: S

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0.

In [198]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 58


#create a dictionary

tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)

#corpus.shape
tokenizer.fit_on_texts(train_corpus)
tokenizer.texts_to_sequences(train_corpus)

x1_train = tokenizer \
    .texts_to_sequences(train_sen1)
x2_train = tokenizer \
    .texts_to_sequences(train_sen2)

print(tokenizer.document_count)

#corpus.shape
tokenizer.fit_on_texts(dev_corpus)
tokenizer.texts_to_sequences(dev_corpus)
x1_dev = tokenizer \
    .texts_to_sequences(dev_sen1)
x2_dev = tokenizer \
    .texts_to_sequences(dev_sen2)

print(tokenizer.document_count)



word_index = tokenizer.word_index




11416
12490


In [199]:


x1_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)





x1_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)








In [52]:
## More code adapted from the keras reference (https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py)
# prepare embedding matrix 
from keras.layers import Embedding
from keras.initializers import Constant

## EMBEDDING_DIM =  ## seems to need to match the embeddings_index dimension
EMBEDDING_DIM = 300
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word) ## This references the loaded embeddings dictionary
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [18]:
import os
import argparse

import pandas as pd
import numpy as np

import random
import tensorflow as tf
import numpy as np
import json as json

from keras import backend as Kbackend

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Flatten, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Bidirectional, LSTM, GRU, TimeDistributed
from keras.layers import Input, concatenate, add
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K



gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
tf.keras.backend.set_session(sess)

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.

    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.

    Note: The layer has been tested with Keras 2.0.6

    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def dot_product(self, x, kernel):
        """
        Wrapper for dot product operation, in order to be compatible with both
        Theano and Tensorflow
        Args:
            x (): input
            kernel (): weights
        Returns:
        """
        if K.backend() == 'tensorflow':
            return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
        else:
            return K.dot(x, kernel)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = self.dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        
        #ait = K.dot(uit, self.u)
        ait = self.dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [19]:
class HAttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(HAttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(HAttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])



In [20]:
#list of numpy arrays as input


In [125]:
concat_train=[]
for x in range(len(x1_train)):
    concat_train.append(np.concatenate((x1_train[x], x2_train[x])))    #5708



In [37]:
concat_dev=[]
for x in range(len(x1_dev)):
    concat_dev.append(np.concatenate((x1_dev[x], x2_dev[x])))         #537

In [147]:
concat_train_l=[]
concat_dev_l=[]

for x in range(len(train_label1)):
    
    concat_train_l.append(np.concatenate((train_label1[x], train_label2[x])))


for x in range(len(dev_label1)):
    
    concat_dev_l.append(np.concatenate((dev_label1[x], dev_label2[x])))


In [32]:

train_l=list(np.array(concat_train_l).reshape(5708, 580))
dev_l=list(np.array(concat_dev_l).reshape(537,580))

In [212]:


from keras import backend as K

#this is arbitrary?


NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = EMBEDDING_DIM

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

top_embedded = embedding_layer(
    top_input)

bm_embedded=embedding_layer(bm_input)


BiLSTM = Bidirectional(LSTM(NUM_LSTM_UNITS, return_sequences=True))



top_output = BiLSTM(top_embedded)

bm_output = BiLSTM(bm_embedded)



merged = concatenate(
    [top_output,bm_output], 
    axis=1)


dense =  TimeDistributed(Dense(
    units=5, 
    activation='softmax'))

predictions1=dense(merged)
#predictions2 = dense(merged)


model = Model(
    inputs=[top_input, bm_input], 
    outputs=[predictions1])


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'])

BATCH_SIZE = 72

NUM_EPOCHS = 30

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
mcp_save = ModelCheckpoint('../Phrase_Models_H5/time_dis_v2.h5', save_best_only=True,  verbose=1, monitor='val_loss', mode='min')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 58)           0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           (None, 58)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 58, 300)      3000300     input_22[0][0]                   
                                                                 input_23[0][0]                   
__________________________________________________________________________________________________
bidirectional_13 (Bidirectional (None, 58, 256)      439296      embedding_3[16][0]               
          

In [213]:

history = model.fit(x=[x1_train, x2_train], y=np.array(concat_train_l), batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, callbacks=[earlyStopping,mcp_save], validation_data=([x1_dev, x2_dev], np.array(concat_dev_l)),
    
    shuffle=True
)

Train on 5708 samples, validate on 537 samples
Epoch 1/30
5708/5708 [==============================] - 34s 6ms/step - loss: 0.2703 - categorical_accuracy: 0.9568 - val_loss: 0.1346 - val_categorical_accuracy: 0.9717

Epoch 00001: val_loss improved from inf to 0.13464, saving model to ../Phrase_Models_H5/time_dis_v2.h5
Epoch 2/30
5708/5708 [==============================] - 28s 5ms/step - loss: 0.1629 - categorical_accuracy: 0.9665 - val_loss: 0.1366 - val_categorical_accuracy: 0.9717

Epoch 00002: val_loss did not improve from 0.13464
Epoch 3/30
5708/5708 [==============================] - 28s 5ms/step - loss: 0.1627 - categorical_accuracy: 0.9665 - val_loss: 0.1344 - val_categorical_accuracy: 0.9717

Epoch 00003: val_loss improved from 0.13464 to 0.13439, saving model to ../Phrase_Models_H5/time_dis_v2.h5
Epoch 4/30
5708/5708 [==============================] - 28s 5ms/step - loss: 0.1625 - categorical_accuracy: 0.9665 - val_loss: 0.1351 - val_categorical_accuracy: 0.9717

Epoch 00004:

In [322]:

for x in range (len(train_label1)):
    zz=train_label2[x]+
    new.append(zz)
new=np.asarray(new)

In [416]:
concat_train=[]
for x in range (len(train_sen1)):
    concat_train.append(np.vstack((train_sen1[x], train_sen2[x])))
concat_train[0]

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [325]:
x1_label_d=x1_label_d.tolist()
x2_label_d=x2_label_d.tolist()
new=[]
for x in range (len(x1_label_d)):
    zz=(x1_label_d[x]+x2_label_d[x])
    new2.append(zz)
new2=np.asarray(new2)




AttributeError: 'list' object has no attribute 'tolist'

In [335]:

history = model.fit(x=[x1_train, x2_train], y=[train_label1, train_label2], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, callbacks=[earlyStopping,mcp_save], validation_data=([x1_dev,x2_dev], [dev_label1, dev_label2]),
    
    shuffle=True
)

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 2 arrays: [array([[[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        ...,
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]],

    ...

In [268]:
history = model.fit(x=[x1_train, x2_train], y=[x1_label_t, x2_label_t], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, callbacks=[earlyStopping,mcp_save], validation_data=([x1_dev,x2_dev],[x1_label_d, x2_label_d]),
    
    shuffle=True
)

Train on 5708 samples, validate on 537 samples
Epoch 1/30
5708/5708 [==============================] - 31s 5ms/step - loss: 0.0606 - dense_26_loss: 0.0255 - dense_26_acc: 0.0801 - dense_26_acc_1: 0.0879 - val_loss: 0.0363 - val_dense_26_loss: 0.0112 - val_dense_26_acc: 0.0577 - val_dense_26_acc_1: 0.0801

Epoch 00001: val_loss improved from inf to 0.03632, saving model to ../Phrase_Models_H5/big_merge_attention.h5
Epoch 2/30
5708/5708 [==============================] - 25s 4ms/step - loss: 0.0590 - dense_26_loss: 0.0248 - dense_26_acc: 0.1228 - dense_26_acc_1: 0.1312 - val_loss: 0.0367 - val_dense_26_loss: 0.0111 - val_dense_26_acc: 0.0633 - val_dense_26_acc_1: 0.1080

Epoch 00002: val_loss did not improve from 0.03632
Epoch 3/30
5708/5708 [==============================] - 25s 4ms/step - loss: 0.0554 - dense_26_loss: 0.0234 - dense_26_acc: 0.1869 - dense_26_acc_1: 0.1910 - val_loss: 0.0365 - val_dense_26_loss: 0.0112 - val_dense_26_acc: 0.0596 - val_dense_26_acc_1: 0.1527

Epoch 00003

In [210]:
#BiLSTM without attention


#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = EMBEDDING_DIM

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    )

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    )


top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)

BiLSTM = Bidirectional(LSTM(NUM_LSTM_UNITS, return_sequences=True))



top_output = BiLSTM(top_embedded)

bm_output = BiLSTM(bm_embedded)


  

dense =  Dense(
    units=290, 
    activation='sigmoid')

predictions1 = dense(top_output)

predictions2 = dense(bm_output)


model = Model(
    inputs=[top_input, bm_input], 
    outputs=[predictions1, predictions2])


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'])

BATCH_SIZE = 72

NUM_EPOCHS = 30

earlyStopping = EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='min')
mcp_save = ModelCheckpoint('../Phrase_Models_H5/big_merge.h5', save_best_only=True,  verbose=1, monitor='val_loss', mode='min')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 58)           0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, 58)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 58, 300)      3000300     input_20[0][0]                   
                                                                 input_21[0][0]                   
__________________________________________________________________________________________________
bidirectional_12 (Bidirectional (None, 58, 256)      439296      embedding_3[14][0]               
          

(5708, 58, 5)

In [211]:
history = model.fit(x=[x1_train, x2_train], y=[train_label1, train_label2], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, callbacks=[earlyStopping,mcp_save], validation_data=([x1_dev, x2_dev],[dev_label1, dev_label2]),
    
    shuffle=True
)

ValueError: Error when checking target: expected dense_12 to have shape (58, 290) but got array with shape (58, 5)

In [47]:

model.save("BiLSTM_PAWS_QQP.h5")        ##first attempt
print("Saved model to disk")  

Saved model to disk


In [214]:
model = load_model("../Phrase_Models_H5/time_dis.h5")


In [215]:

predicts = model.predict(
    [x1_train, x2_train])

In [177]:
print(predicts[0][22])
print(concat_train_l[0][22])

[0.97118807 0.0070615  0.00425047 0.0125009  0.00499898]
[1. 0. 0. 0. 0.]


array([0., 0., 0., 1., 0.], dtype=float32)

In [205]:
                #10000: O, 01000: B, 00100: I, 00010: E, 00001: S
dicti={}
errr=[]

for x in range(len(predicts[4])):    #sentence number
    
    for y in range(len(predicts[4][x])):       #which word in the sentence
        
        if ((np.argmax(predicts[x][y]))!=(np.argmax(concat_train_l[x][y]))):
            errr.append(y)
            
    dicti[x]=errr

['for',
 'their',
 'performances',
 'in',
 'the',
 'game',
 ',',
 'quarterback',
 'jameis',
 'winston',
 'and',
 'defensive',
 'back',
 'p.',
 'j.',
 'williams',
 'were',
 'named',
 'the',
 'game',
 "'s",
 'most',
 'valuable',
 'players',
 '.']

In [296]:
                #10000: O, 01000: B, 00100: I, 00010: E, 00001: S
    
dicti={}    #the sentence and word of error
errs = []
truth=[]
sen1=[]
sen2=[]
id_sent=[]
id_word=[]

for x in range(len(predicts)): 
    errr=[]
    for y in range(len(predicts[x])):
        
        if (np.argmax(predicts[x][y]))!=(np.argmax(concat_train_l[x][y])):
            
            truth.append(concat_train_l[x][y])
            errs.append(predicts[x][y])
            errr.append(y)
            id_sent.append(x)
            id_word.append(y)
            
            sen1.append(train_sen1[x])
            sen2.append(train_sen2[x])
            
    dicti[x]=errr

In [355]:
err_ana.column(4)[0]

['for',
 'their',
 'performances',
 'in',
 'the',
 'game',
 ',',
 'quarterback',
 'jameis',
 'winston',
 'and',
 'defensive',
 'back',
 'p.',
 'j.',
 'williams',
 'were',
 'named',
 'the',
 'game',
 "'s",
 'most',
 'valuable',
 'players',
 '.']

In [348]:
(err_ana.column(0)[22178])

5707

In [343]:
err_ana.column(4)[err_ana.column(0)[22178]][err_ana.column(1)[22178]]

IndexError: list index out of range

In [351]:
sen1_err=[]
sen2_err=[]
for x in range (err_ana.num_rows):
    if err_ana.column(1)[x]<58:   #in the first sentence
        sen1_err.append(train_sen1[err_ana.column(0)[x]][err_ana.column(1)[x]])
        sen2_err.append(0)
    else:
        sen1_err.append(0)
        sen2_err.append(train_sen2[err_ana.column(0)[x]][(err_ana.column(1)[x]-58)])

In [352]:
err_analy = Table().with_columns("id_sent", id_sent, "id_word", id_word, "ground truth", truth, "predictions", errs, "sen1", sen1_err, 'sen2', sen2_err)
err_analy.show(30)

id_sent,id_word,ground truth,predictions,sen1,sen2
0,20,[0. 1. 0. 0. 0.],[0.9660126 0.00872339 0.00455601 0.01459314 0.00611483],'s,0
0,21,[0. 0. 0. 1. 0.],[0.96925867 0.00768918 0.00430224 0.01327901 0.00547091],most,0
0,70,[0. 0. 0. 0. 1.],[0.9187401 0.02769896 0.0095076 0.03080195 0.01325135],0,most
1,2,[0. 1. 0. 0. 0.],[0.9040129 0.04169884 0.01086036 0.02902739 0.01440052],are,0
1,3,[0. 0. 0. 1. 0.],[0.90196157 0.04200785 0.01101631 0.03033178 0.01468245],generally,0
1,60,[0. 1. 0. 0. 0.],[0.90401304 0.04169881 0.01086028 0.02902741 0.01440049],0,are
1,61,[0. 0. 0. 1. 0.],[0.9019618 0.0420078 0.01101622 0.03033179 0.0146824 ],0,usually
2,2,[0. 1. 0. 0. 0.],[0.9040115 0.04170053 0.01085953 0.02902849 0.01439998],taken,0
2,3,[0. 0. 1. 0. 0.],[0.90195966 0.0420099 0.0110154 0.0303332 0.0146818 ],part,0
2,4,[0. 0. 0. 1. 0.],[0.90050465 0.04191463 0.01113533 0.03152322 0.01492213],in,0


In [208]:
dicti

{0: [20, 21, 70],
 1: [2, 3, 60, 61],
 2: [2, 3, 4, 60, 61],
 3: [14, 15, 73],
 4: [15, 16, 75],
 5: [4, 5, 83],
 6: [15, 73, 74],
 7: [6, 7, 70],
 8: [10, 11, 67],
 9: [9, 10, 70],
 10: [1, 2, 62],
 11: [5, 6, 63],
 12: [1, 2, 60, 61],
 13: [4, 5, 6, 63, 64],
 14: [23, 24, 83],
 15: [4, 16, 62, 63],
 16: [9, 10, 68, 69],
 17: [10, 11, 74],
 18: [23, 24, 71],
 19: [1, 2, 60],
 20: [8, 9, 10, 59, 69],
 21: [7, 8, 66],
 22: [7, 8, 67],
 23: [15, 16, 17, 76],
 24: [11, 12, 13, 71, 72],
 25: [15, 16, 64, 73, 74],
 26: [1, 2, 59],
 27: [7, 8, 9, 65],
 28: [10, 70, 71],
 29: [27, 28, 80],
 30: [13, 14, 73],
 31: [7, 8, 78],
 32: [7, 8, 65],
 33: [10, 11, 67],
 34: [6, 7, 65],
 35: [6, 7, 59, 60],
 36: [15, 16, 67],
 37: [5, 6, 64],
 38: [1, 2, 60],
 39: [23, 24, 82, 83, 84],
 40: [4, 5, 61, 62],
 41: [1, 8, 9, 66],
 42: [2, 3, 60, 61],
 43: [13, 14, 15, 78, 79],
 44: [8, 9, 10, 66],
 45: [9, 10, 73],
 46: [6, 62, 63],
 47: [4, 70, 71],
 48: [10, 11, 67],
 49: [3, 4, 60, 61],
 50: [11, 12, 13

In [16]:
error_analysis = Table()


error_analysis = error_analysis.with_column("id", no, "sent1", err1, "sent2", err2, "label", wrongCat)


#false POSITIVE: swap two words
#false negative: swap two specific nouns
error_analysis.show(30)

id,sent1,sent2,label
1,b'How can you treat ocd ? Is there any helpful suggestio ...,b'How can you treat OCD ? Is there any helpful suggestio ...,1
7,b'What are some beautiful lines to comment on Beautiful ...,b'What are some Beautiful lines to comment on beautiful ...,1
8,b'Can small dogs breed with large dogs ?',b'Can small dogs breed with large dogs ?',1
13,b'Are Indian IT services companies like Tech Mahindra/ W ...,b'Are Indian IT services companies like Wipro/TCS/Tech . ...,1
14,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ? ''""",1
15,b'What should I solve and what or how should I revise fo ...,b'What should I revise and what or how should I solve fo ...,0
20,b'How is the relative ratio of brain waves ( alpha/beta/ ...,b'How is the ratio of relative brain waves ( alpha/beta/ ...,0
21,b'How can I make an android app using Python 3 ? Is ther ...,b'How can I develop an android app using Python 3 ? Is t ...,1
24,b'Headbreak : What can I do to prevent heart break night ...,b'Headbreak : What can I do to prevent heart break night ...,1
25,b'When is latent heat positive ?',b'When is latent heat positive ?',1


id,sent1,sent2,label
1,b'How can you treat ocd ? Is there any helpful suggestio ...,b'How can you treat OCD ? Is there any helpful suggestio ...,1
7,b'What are some beautiful lines to comment on Beautiful ...,b'What are some Beautiful lines to comment on beautiful ...,1
8,b'Can small dogs breed with large dogs ?',b'Can small dogs breed with large dogs ?',1
13,b'Are Indian IT services companies like Tech Mahindra/ W ...,b'Are Indian IT services companies like Wipro/TCS/Tech . ...,1
14,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ? ''""",1
21,b'How can I make an android app using Python 3 ? Is ther ...,b'How can I develop an android app using Python 3 ? Is t ...,1
24,b'Headbreak : What can I do to prevent heart break night ...,b'Headbreak : What can I do to prevent heart break night ...,1
25,b'When is latent heat positive ?',b'When is latent heat positive ?',1
28,"b""What does the phrase 'Do you feel me ' mean ?""","b""What does the phrase `Do you feel me'mean ?""",1
29,b'What are some unexpected things first-time visitors to ...,"b'What are some unexpected things , first-time visitors ...",1
